In [1]:
# Validate there is exactly ONE empty sales invoice form, and return it.

import json, os
from pymongo import MongoClient
from dotenv import load_dotenv

# --- Setup ---
load_dotenv()
uri = os.getenv("MONGODB_URI")
database_name = os.getenv("MONGODB_DATABASE")
collection_name = "sales_invoices"

if not uri or not database_name:
    raise RuntimeError("Missing MONGODB_URI or MONGODB_DATABASE in environment.")

client = MongoClient(uri)
db = client[database_name]
col = db[collection_name]

# --- Definition of "empty form" ---
empty_filter = {
    "$or": [
        {"items": {"$exists": False}},
        {"items": {"$size": 0}}
    ]
}

# --- Find ALL empty forms (newest first for easier debugging) ---
empty_docs = list(col.find(empty_filter).sort("created", -1))

count = len(empty_docs)

if count == 0:
    raise RuntimeError("❌ Validation failed: No EMPTY form found in collection.")

if count > 1:
    # Provide a brief debug summary so you can resolve duplicates.
    summary = [
        {
            "_id": str(d.get("_id")),
            "id": d.get("id"),
            "current_form_qr_code": d.get("current_form_qr_code"),
            "created": str(d.get("created")),
            "date": d.get("date"),
        }
        for d in empty_docs
    ]
    raise RuntimeError(
        "❌ Validation failed: More than one EMPTY form found. "
        f"Count={count}. Resolve duplicates so only one remains.\n"
        f"Debug summary:\n{json.dumps(summary, indent=2, default=str)}"
    )

# --- Exactly one empty form -> return/print it ---
empty_doc = empty_docs[0]
print("✅ Validation passed: Exactly ONE EMPTY form found.")
print(json.dumps(empty_doc, indent=2, default=str))

# 'empty_doc' is now available for further notebook use.


✅ Validation passed: Exactly ONE EMPTY form found.
{
  "_id": "68ed1b27c3992aaea60563f1",
  "id": "5812d170-c0fb-4a77-a0f1-263616dc7a67",
  "previous_form_qr_code": "20251012",
  "current_form_qr_code": "a1b36b5a",
  "tin": "123-456-789",
  "location": "JEF Gas Station \u2013 Sikatuna Branch",
  "created": "2025-10-13T15:30:47.572217+00:00",
  "date": "2025-10-13"
}
